###### 【問題1】コードレビュー
転移学習を使用してセグメンテーションの精度を改善したコードを提示するので、レビューを行ってください。
視点例
Sprint20で使用した実装とはどのように違うのか
転移学習をどのように行っているか

###### Kaggle kernel 

In [ ]:
from keras import Model
from keras.layers import (Activation, BatchNormalization, Concatenate, Conv2D,
                          Conv2DTranspose, Dropout, Input, MaxPooling2D,
                          UpSampling2D, concatenate)
from keras.optimizers import Adam


"""
U-Netのブロック
"""
def conv_block(m, dim, acti, bn, res, do=0):
    n = Conv2D(dim, 3, activation=acti, padding='same')(m)
    n = BatchNormalization()(n) if bn else n
    n = Dropout(do)(n) if do else n
    n = Conv2D(dim, 3, activation=acti, padding='same')(n)
    n = BatchNormalization()(n) if bn else n
    return Concatenate()([m, n]) if res else n


"""
・Depth分だけlevel_blockを再起呼び出し
・再起呼び出しのたびにConv_blockで定義されるネットワークを追加、Conv_blockの後にMaxpoolingかConvolutionを行う
・depth=0に到達した後、Up-sampling+Conv2D (or Conv2DTranspose)　+ Concatenate + conv_block がDepthの数だけ追加される
・最終層(Depth=0)はConv_block (else処理)

m : Input 
dim : チャンネル方向の次元
depth :　U-Netの深さ（同じ識別ネットワークを何回繰り返すか）
inc :　各深さでのチャンネル方向の次元が増分（前の深さの何倍になるか）
acti :　活性化関数
do : Dropoutの比率
bn : Batch normalizationの有無
mp : Maxpoolingを使うか、Convolutionをするか 
up :　デコーダーのup samplingの方法( up-sampling+Conv2D or Conv2DTranspose )
res : 使われてない？
"""
def level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
    if depth > 0:
        """
        エンコーダー
        """
        n = conv_block(m, dim, acti, bn, res)
        m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
        m = level_block(m, int(inc * dim), depth - 1,
                        inc, acti, do, bn, mp, up, res)
        
        """
        デコーダー
        """
        if up:
            m = UpSampling2D()(m)
            m = Conv2D(dim, 2, activation=acti, padding='same')(m)
        else:
            m = Conv2DTranspose(dim, 3, strides=2,
                                activation=acti, padding='same')(m)
        n = Concatenate()([n, m])
        m = conv_block(n, dim, acti, bn, res)
    else:
        """
        最終層(depth=0)
        """
        m = conv_block(m, dim, acti, bn, res, do)
    return m


"""
U-NET アーキテクチャを定義
"""
def UNet(params):

    img_shape = params['input_dim']
    out_ch = 1
    start_ch = 8
    depth = 3
    inc_rate = 2.
    activation = 'relu'
    dropout = 0.5
    batchnorm = False
    maxpool = True
    upconv = True
    residual = False

    """
    入力
    """
    i = Input(shape=img_shape)
    """
    U-NETコア
    """
    o = level_block(i, start_ch, depth, inc_rate, activation,
                    dropout, batchnorm, maxpool, upconv, residual)
    
    """
    出力層
    """
    o = Conv2D(out_ch, 1)(o)
    # Sigmoid activation is used because model is trained with binary_crossentropy.
    o =  Activation('sigmoid')(o)

    model = Model(inputs=i, outputs=o)
    
    return model

###### ResNet 転移学習

In [ ]:
def unet_resnet(input_size, decoder_block,
                weights='imagenet',
                loss_func='binary_crossentropy',
                metrics_list=[my_iou_metric],
                use_lovash=False):

    """
    エンコーダにResNetを使用
    weights='imagenet'でImageNetの学習済み重みを設定
    """
    # Base model - encoder
    base_model = ResNet50(
        input_shape=input_size, 
        include_top=False,
        weights=weights)
    
    """
    Skip connectionのため対応するレイヤーの出力を取り出し
    """
    # Layers for feature extraction in the encoder part
    encoder1 = base_model.get_layer('activation_1').output
    encoder2 = base_model.get_layer('activation_10').output
    encoder3 = base_model.get_layer('activation_22').output
    encoder4 = base_model.get_layer('activation_40').output
    encoder5 = base_model.get_layer('activation_49').output

    """
    最深部
    """
    # Center block
    center = decoder_block(
        encoder5, 'center', num_filters=512)
    concat5 = concatenate([center, encoder5], axis=-1) #なぞ？ Upsampling無しにconcatしている。

    """
    デコーダ
    """
    # Decoder part.
    # Every decoder block processed concatenated output from encoder and decoder part.
    # This creates skip connections.
    # Afterwards, decoder output is upsampled to dimensions equal to encoder output part.
    decoder4 = decoder_block(
        concat5, 'decoder4', num_filters=256)
    concat4 = concatenate([UpSampling2D()(decoder4), encoder4], axis=-1)

    decoder3 = decoder_block(
        concat4, 'decoder3', num_filters=128)
    concat3 = concatenate([UpSampling2D()(decoder3), encoder3], axis=-1)

    decoder2 = decoder_block(
        concat3, 'decoder2', num_filters=64)
    concat2 = concatenate([UpSampling2D()(decoder2), encoder2], axis=-1)

    decoder1 = decoder_block(
        concat2, 'decoder1', num_filters=64)
    concat1 = concatenate([UpSampling2D()(decoder1), encoder1], axis=-1)

    """
    出力層
    """
    # Final upsampling and decoder block for segmentation.
    output = UpSampling2D()(concat1)
    output = decoder_block(
        output, 'decoder_output', num_filters=32)
    output = Conv2D(
        1, (1, 1), activation=None, name='prediction')(output)
    if not use_lovash:
        output = Activation('sigmoid')(output)
    
    """
    モデルのアセンブル 
    """
    model = Model(base_model.input, output)
    model.compile(loss=loss_func, optimizer='adam', metrics=metrics_list)

    return model

###### 【問題2】コードの書き換え
エンコーダーにResNetが使用されていたコードをVGGに変更してください。

In [ ]:
def unet_resnet(input_size, decoder_block,
                weights='imagenet',
                loss_func='binary_crossentropy',
                metrics_list=[my_iou_metric],
                use_lovash=False):
    
    inputs = Input(shape=input_size)
    
    """
    エンコーダとしてVGG19を設定
    weights='imagenet'でImageNetの学習済み重みを設定
    """
    base_model = VGG19(
       weights=weights,
       include_top=False,
       input_shape=input_size)
    
    #VGG19層凍結設定
    for layer in base_model.layers:
      layer.trainable = True
    
    """
    対応するデコーダの層のサイズからVGGどの層の出力をContructing pathとするか決める
    """
    # Layers for feature extraction in the encoder part
    encoder1 = base_model.get_layer('block1_pool').output
    encoder2 = base_model.get_layer('block3_conv4').output
    encoder3 = base_model.get_layer('block4_conv4').output
    encoder4 = base_model.get_layer('block5_conv4').output
    encoder5 = base_model.get_layer('block5_pool').output
    #Center block
    center = decoder_block(
        encoder5, 'center', num_filters=512)
        
    concat5 = concatenate([center, encoder5], axis=-1)
    # Decoder part.
    # Every decoder block processed concatenated output from encoder and decoder part.
    # This creates skip connections.
    # Afterwards, decoder output is upsampled to dimensions equal to encoder output part.
    decoder4 = decoder_block(
        concat5, 'decoder4', num_filters=256)
    concat4 = concatenate([UpSampling2D()(decoder4), encoder4], axis=-1)

    decoder3 = decoder_block(
        concat4, 'decoder3', num_filters=128)
    concat3 = concatenate([UpSampling2D()(decoder3), encoder3], axis=-1)

    decoder2 = decoder_block(
        concat3, 'decoder2', num_filters=64)
    concat2 = concatenate([UpSampling2D()(decoder2), encoder2], axis=-1)

    decoder1 = decoder_block(
        concat2, 'decoder1', num_filters=64)
    concat1 = concatenate([UpSampling2D()(decoder1), encoder1], axis=-1)

    # Final upsampling and decoder block for segmentation.
    output = UpSampling2D()(concat1)
    output = decoder_block(
        output, 'decoder_output', num_filters=32)
    output = Conv2D(
        1, (1, 1), activation=None, name='prediction')(output)
    if not use_lovash:
        output = Activation('sigmoid')(output) 
    
    model = Model(base_model.inputs, output)
    model.compile(loss=loss_func, optimizer='adam', metrics=metrics_list)

    return model

###### 【問題3】学習・推定
ResNetとVGG双方のコードで学習・推定を行い、結果を比較してください。

Epoch=40, batch=16で学習

###### ResNet

<img src="ResNet/table.png">

<img src="ResNet/graph.png">

###### VGG (w/o training)

<img src="VGG19_wo_training/table.png">

<img src="VGG19_wo_training/graph.png">

###### VGG (w training)

<img src="VGG19_w_training/table.png">

<img src="VGG19_w_training/graph.png">

結果、ResNetが最も良い結果となった。VGG19を用いた転移学習では層の凍結をしないほうが良い結果となった。